In [1]:
# MCP Tools in Jupyter Notebooks
# ------------------------------
# IMPORTANT: When using MCP tools in Jupyter notebooks, HTTP-based MCP servers are recommended.
# Stdio-based MCP servers (MCPStdioParams) are now supported with compatibility patches,
# but may have limited functionality. You'll see a warning when using them in Jupyter.

import asyncio
import railtracks as rt
from railtracks.nodes.library.easy_usage_wrappers.mcp_tool import from_mcp_server
from railtracks.rt_mcp import MCPHttpParams, MCPStdioParams



In [2]:
# Example 1: Using HTTP-based MCP servers (RECOMMENDED for Jupyter)
# -----------------------------------------------------------------
# HTTP-based MCP servers work well in Jupyter notebooks and should be used whenever possible
fetch_server = from_mcp_server(MCPHttpParams(url="https://remote.mcpservers.org/fetch/mcp"))
fetch_tools = fetch_server.tools

Attempting to import get_ipython from IPython...


In [3]:
# Example 2: Using stdio-based MCP servers (NOW SUPPORTED with compatibility patches)
# ---------------------------------------------------------------------------------
# With the new Jupyter compatibility patches, stdio-based MCP servers now work in Jupyter notebooks
# You'll see a warning about potential limitations, but the code should run successfully

# Define the command and arguments for the stdio-based MCP server
# Note: You'll need to adjust these paths for your environment
MCP_COMMAND = "uvx"  # Path to your executable
MCP_ARGS = ["mcp-server-time"]                      # Arguments for the executable

# Create the stdio-based MCP server
# This will show a warning but should work with the compatibility patches
time_stdio_server = from_mcp_server(MCPStdioParams(command=MCP_COMMAND, args=MCP_ARGS))
time_stdio_tools = time_stdio_server.tools


Attempting to import get_ipython from IPython...


In [4]:
# Create a parent tool that can use all the tools from both HTTP and stdio MCP servers
parent_tool = rt.library.tool_call_llm(
    connected_nodes={*fetch_tools, *time_stdio_tools},
    pretty_name="Parent Tool",
    system_message=rt.llm.SystemMessage("Provide a response using the tool when asked."),
    llm_model=rt.llm.OpenAILLM("gpt-4o"),
)


C:\Users\Levi\PycharmProjects\rc\src\railtracks\exceptions\node_creation\validation.py:219: UserWarning: You have set max_tool_calls to None. The llm will be able to make unlimited tool calls.
  warnings.warn(


In [5]:
user_message = ("Tell me about conductr.ai. Then, tell me what time it is.")

with rt.Runner(executor_config=rt.ExecutorConfig(logging_setting="QUIET", timeout=1000)) as runner:
    message_history = rt.llm.MessageHistory(
       [
            rt.llm.UserMessage(
                user_message
            )
        ]
    )
    response = await rt.call(parent_tool, user_input=message_history)

    print("Response:", response)


C:\Users\Levi\PycharmProjects\rc\src\railtracks\exceptions\node_invocation\validation.py:46: RuntimeWarning: You have set max_tool_calls to None. The llm will be able to make unlimited tool calls.
  warnings.warn(


Response: assistant: **Conductr.ai Overview:**
Conductr.ai is focused on enhancing developer productivity by integrating Developer Productivity Engineering (DPE) principles. It aims to streamline software development through automation, smart insights, and efficient CI/CD processes. Conductr helps developers shift left by integrating quality, security, and performance early into the development cycle. Key features include:
- **Root Cause Agent:** Provides AI-powered alerts directly in the Integrated Development Environment (IDE) to surface errors and dependencies quickly.
- **Development Team Tools:** Automatically generates daily standup updates, tracks errors, enhances team communication, and provides deployment insights.
- **Velocity Dashboard:** Offers a central hub for tracking key data, making it easier for teams to identify areas for process improvement and make data-driven decisions.

For more information, you can visit their official [website](https://conductr.ai).

**Current 